In [2]:
import pandas as pd
import numpy as np

In [3]:
region_names = pd.read_csv("./datasets/raw/region_data/region_names.csv")
region_names.head()

nm_en                    nm_fr   ab_en     ab_fr  \
0  Newfoundland and Labrador  Terre-Neuve-et-Labrador    N.L.  T.-N.-L.   
1       Prince Edward Island    Île-du-Prince-Édouard  P.E.I.  Î.-P.-É.   
2                Nova Scotia          Nouvelle-Écosse    N.S.     N.-É.   
3              New Brunswick        Nouveau-Brunswick    N.B.     N.-B.   
4                     Quebec                   Québec    Que.        Qc   

  al_code  nu_code iso_code  
0      NL       10    CA-NL  
1      PE       11    CA-PE  
2      NS       12    CA-NS  
3      NB       13    CA-NB  
4      QC       24    CA-QC

In [403]:
region_names = region_names.rename(columns={'nm_en': 'Name_EN', 
                             'nm_fr': 'Name_FR', 
                             'ab_en': 'Abbrev_EN', 
                             'ab_fr': 'Abbrev_FR',
                             'al_code': 'AlphaCode',
                             'nu_code': 'NumericCode'})
region_names.head()

Name_EN                  Name_FR Abbrev_EN Abbrev_FR  \
0  Newfoundland and Labrador  Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1       Prince Edward Island    Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                Nova Scotia          Nouvelle-Écosse      N.S.     N.-É.   
3              New Brunswick        Nouveau-Brunswick      N.B.     N.-B.   
4                     Quebec                   Québec      Que.        Qc   

  AlphaCode  NumericCode iso_code  
0        NL           10    CA-NL  
1        PE           11    CA-PE  
2        NS           12    CA-NS  
3        NB           13    CA-NB  
4        QC           24    CA-QC

In [404]:
region_names = region_names.drop(columns=['iso_code'])
region_names.head()

Name_EN                  Name_FR Abbrev_EN Abbrev_FR  \
0  Newfoundland and Labrador  Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1       Prince Edward Island    Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                Nova Scotia          Nouvelle-Écosse      N.S.     N.-É.   
3              New Brunswick        Nouveau-Brunswick      N.B.     N.-B.   
4                     Quebec                   Québec      Que.        Qc   

  AlphaCode  NumericCode  
0        NL           10  
1        PE           11  
2        NS           12  
3        NB           13  
4        QC           24

Adding classes

In [405]:
region_class = pd.read_csv("./datasets/raw/region_data/region_classification.csv", encoding='latin-1')
region_class.head()

Level         Hierarchical structure     Code                Class title
0      1  Geographical region of Canada        1                   Atlantic
1      2         Province and territory       10  Newfoundland and Labrador
2      3                Census division     1001             Division No. 1
3      4             Census subdivision  1001101    Division No. 1, Subd. V
4      4             Census subdivision  1001105        Portugal Cove South

In [406]:
region_class_subset_greg = region_class[region_class['Level'] == 1]
region_class_subset_prov = region_class[region_class['Level'] == 2]
display(region_class_subset_greg.head())
display(len(region_class_subset_greg))
display(region_class_subset_prov.head())
display(len(region_class_subset_prov))

Level         Hierarchical structure  Code       Class title
0         1  Geographical region of Canada     1          Atlantic
905       1  Geographical region of Canada     2            Quebec
2290      1  Geographical region of Canada     3           Ontario
2916      1  Geographical region of Canada     4          Prairies
4584      1  Geographical region of Canada     5  British Columbia

6

Level  Hierarchical structure  Code                Class title
1        2  Province and territory    10  Newfoundland and Labrador
385      2  Province and territory    11       Prince Edward Island
501      2  Province and territory    12                Nova Scotia
616      2  Province and territory    13              New Brunswick
906      2  Province and territory    24                     Quebec

13

In [407]:
region_class_subset_prov['GroupCode'] = [int(i/10) for i in region_class_subset_prov['Code']]
display(region_class_subset_prov.head())

C:\Users\svetl\AppData\Local\Temp\ipykernel_4620\2780356622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_class_subset_prov['GroupCode'] = [int(i/10) for i in region_class_subset_prov['Code']]


Level  Hierarchical structure  Code                Class title  GroupCode
1        2  Province and territory    10  Newfoundland and Labrador          1
385      2  Province and territory    11       Prince Edward Island          1
501      2  Province and territory    12                Nova Scotia          1
616      2  Province and territory    13              New Brunswick          1
906      2  Province and territory    24                     Quebec          2

In [408]:
# test
region_class_subset_greg[region_class_subset_greg['Code']==1]['Class title'].iloc[0]

'Atlantic'

In [409]:
region_class_subset_prov['Group'] = [
    region_class_subset_greg[region_class_subset_greg['Code']==code]['Class title'].iloc[0]
    for code in region_class_subset_prov['GroupCode']
]
display(region_class_subset_prov.head())

C:\Users\svetl\AppData\Local\Temp\ipykernel_4620\604966975.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_class_subset_prov['Group'] = [


Level  Hierarchical structure  Code                Class title  \
1        2  Province and territory    10  Newfoundland and Labrador   
385      2  Province and territory    11       Prince Edward Island   
501      2  Province and territory    12                Nova Scotia   
616      2  Province and territory    13              New Brunswick   
906      2  Province and territory    24                     Quebec   

     GroupCode     Group  
1            1  Atlantic  
385          1  Atlantic  
501          1  Atlantic  
616          1  Atlantic  
906          2    Quebec

In [410]:
region_combined = region_names.copy(deep=True)
region_combined['GeographicRegion'] = [
    region_class_subset_prov[region_class_subset_prov['Class title'] == name]['Group'].iloc[0]
    for name in region_combined['Name_EN']
]
region_combined.head(10)

Name_EN                  Name_FR Abbrev_EN Abbrev_FR  \
0  Newfoundland and Labrador  Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1       Prince Edward Island    Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                Nova Scotia          Nouvelle-Écosse      N.S.     N.-É.   
3              New Brunswick        Nouveau-Brunswick      N.B.     N.-B.   
4                     Quebec                   Québec      Que.        Qc   
5                    Ontario                  Ontario      Ont.      Ont.   
6                   Manitoba                 Manitoba      Man.      Man.   
7               Saskatchewan             Saskatchewan     Sask.     Sask.   
8                    Alberta                  Alberta     Alta.      Alb.   
9           British Columbia     Colombie-Britannique      B.C.     C.-B.   

  AlphaCode  NumericCode  GeographicRegion  
0        NL           10          Atlantic  
1        PE           11          Atlantic  
2        NS           12          Atlantic  
3        NB           13          Atlantic  
4        QC           24            Quebec  
5        ON           35           Ontario  
6        MB           46          Prairies  
7        SK           47          Prairies  
8        AB           48          Prairies  
9        BC           59  British Columbia

In [411]:
regions_add = pd.DataFrame(columns = region_combined.columns)
regions_add['Name_EN'] = [
    name
    for name in region_class_subset_greg['Class title'].values
    if not(name in region_class_subset_prov['Class title'].values)
]
display(regions_add.head())

Name_EN Name_FR Abbrev_EN Abbrev_FR AlphaCode NumericCode  \
0     Atlantic     NaN       NaN       NaN       NaN         NaN   
1     Prairies     NaN       NaN       NaN       NaN         NaN   
2  Territories     NaN       NaN       NaN       NaN         NaN   

  GeographicRegion  
0              NaN  
1              NaN  
2              NaN

In [412]:
regions_add = regions_add.assign(GeographicRegion='Canada')

regions_add['NumericCode'] = [
    region_class_subset_greg[region_class_subset_greg['Class title'] == name]['Code'].iloc[0]
    for name in regions_add['Name_EN']
]
display(regions_add.head())

Name_EN Name_FR Abbrev_EN Abbrev_FR AlphaCode  NumericCode  \
0     Atlantic     NaN       NaN       NaN       NaN            1   
1     Prairies     NaN       NaN       NaN       NaN            4   
2  Territories     NaN       NaN       NaN       NaN            6   

  GeographicRegion  
0           Canada  
1           Canada  
2           Canada

In [413]:
region_combined = pd.concat([region_combined, regions_add])
region_combined.head(16)

Name_EN                    Name_FR Abbrev_EN Abbrev_FR  \
0   Newfoundland and Labrador    Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1        Prince Edward Island      Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                 Nova Scotia            Nouvelle-Écosse      N.S.     N.-É.   
3               New Brunswick          Nouveau-Brunswick      N.B.     N.-B.   
4                      Quebec                     Québec      Que.        Qc   
5                     Ontario                    Ontario      Ont.      Ont.   
6                    Manitoba                   Manitoba      Man.      Man.   
7                Saskatchewan               Saskatchewan     Sask.     Sask.   
8                     Alberta                    Alberta     Alta.      Alb.   
9            British Columbia       Colombie-Britannique      B.C.     C.-B.   
10                      Yukon                      Yukon      Y.T.        Yn   
11      Northwest Territories  Territoires du Nord-Ouest    N.W.T.   T.N.-O.   
12                    Nunavut                    Nunavut      Nvt.        Nt   
0                    Atlantic                        NaN       NaN       NaN   
1                    Prairies                        NaN       NaN       NaN   
2                 Territories                        NaN       NaN       NaN   

   AlphaCode  NumericCode  GeographicRegion  
0         NL           10          Atlantic  
1         PE           11          Atlantic  
2         NS           12          Atlantic  
3         NB           13          Atlantic  
4         QC           24            Quebec  
5         ON           35           Ontario  
6         MB           46          Prairies  
7         SK           47          Prairies  
8         AB           48          Prairies  
9         BC           59  British Columbia  
10        YT           60       Territories  
11        NT           61       Territories  
12        NU           62       Territories  
0        NaN            1            Canada  
1        NaN            4            Canada  
2        NaN            6            Canada

In [414]:
len(region_combined)

16

In [415]:
region_combined = region_combined.reset_index(drop=True)
region_combined.head(16)

Name_EN                    Name_FR Abbrev_EN Abbrev_FR  \
0   Newfoundland and Labrador    Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1        Prince Edward Island      Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                 Nova Scotia            Nouvelle-Écosse      N.S.     N.-É.   
3               New Brunswick          Nouveau-Brunswick      N.B.     N.-B.   
4                      Quebec                     Québec      Que.        Qc   
5                     Ontario                    Ontario      Ont.      Ont.   
6                    Manitoba                   Manitoba      Man.      Man.   
7                Saskatchewan               Saskatchewan     Sask.     Sask.   
8                     Alberta                    Alberta     Alta.      Alb.   
9            British Columbia       Colombie-Britannique      B.C.     C.-B.   
10                      Yukon                      Yukon      Y.T.        Yn   
11      Northwest Territories  Territoires du Nord-Ouest    N.W.T.   T.N.-O.   
12                    Nunavut                    Nunavut      Nvt.        Nt   
13                   Atlantic                        NaN       NaN       NaN   
14                   Prairies                        NaN       NaN       NaN   
15                Territories                        NaN       NaN       NaN   

   AlphaCode  NumericCode  GeographicRegion  
0         NL           10          Atlantic  
1         PE           11          Atlantic  
2         NS           12          Atlantic  
3         NB           13          Atlantic  
4         QC           24            Quebec  
5         ON           35           Ontario  
6         MB           46          Prairies  
7         SK           47          Prairies  
8         AB           48          Prairies  
9         BC           59  British Columbia  
10        YT           60       Territories  
11        NT           61       Territories  
12        NU           62       Territories  
13       NaN            1            Canada  
14       NaN            4            Canada  
15       NaN            6            Canada

In [4]:
region_area = pd.read_csv("./datasets/raw/region_data/region_area.csv")
region_area.head(14)

Region  Area%  Area_skm  Land_skm  Freshwater_skm
0                      Canada  100.0   9984670   9093507        891163.0
1   Newfoundland and Labrador    4.1    405212    373872         31340.0
2        Prince Edward Island    0.1      5660      5660             NaN
3                 Nova Scotia    0.6     55284     53338          1946.0
4               New Brunswick    0.7     72908     71450          1458.0
5                      Quebec   15.4   1542056   1365128        176928.0
6                     Ontario   10.8   1076395    917741        158654.0
7                    Manitoba    6.5    647797    553556         94241.0
8                Saskatchewan    6.5    651036    591670         59366.0
9                     Alberta    6.6    661848    642317         19531.0
10           British Columbia    9.5    944735    925186         19549.0
11                      Yukon    4.8    482443    474391          8052.0
12      Northwest Territories   13.5   1346106   1183085        163021.0
13                    Nunavut   21.0   2093190   1936113        157077.0

In [417]:
region_combined = region_combined.merge(region_area, how="left", left_on='Name_EN', right_on='Region')
region_combined = region_combined.drop(columns=["Region"])
region_combined.head(16)

Name_EN                    Name_FR Abbrev_EN Abbrev_FR  \
0   Newfoundland and Labrador    Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1        Prince Edward Island      Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                 Nova Scotia            Nouvelle-Écosse      N.S.     N.-É.   
3               New Brunswick          Nouveau-Brunswick      N.B.     N.-B.   
4                      Quebec                     Québec      Que.        Qc   
5                     Ontario                    Ontario      Ont.      Ont.   
6                    Manitoba                   Manitoba      Man.      Man.   
7                Saskatchewan               Saskatchewan     Sask.     Sask.   
8                     Alberta                    Alberta     Alta.      Alb.   
9            British Columbia       Colombie-Britannique      B.C.     C.-B.   
10                      Yukon                      Yukon      Y.T.        Yn   
11      Northwest Territories  Territoires du Nord-Ouest    N.W.T.   T.N.-O.   
12                    Nunavut                    Nunavut      Nvt.        Nt   
13                   Atlantic                        NaN       NaN       NaN   
14                   Prairies                        NaN       NaN       NaN   
15                Territories                        NaN       NaN       NaN   

   AlphaCode  NumericCode  GeographicRegion  Area%   Area_skm   Land_skm  \
0         NL           10          Atlantic    4.1   405212.0   373872.0   
1         PE           11          Atlantic    0.1     5660.0     5660.0   
2         NS           12          Atlantic    0.6    55284.0    53338.0   
3         NB           13          Atlantic    0.7    72908.0    71450.0   
4         QC           24            Quebec   15.4  1542056.0  1365128.0   
5         ON           35           Ontario   10.8  1076395.0   917741.0   
6         MB           46          Prairies    6.5   647797.0   553556.0   
7         SK           47          Prairies    6.5   651036.0   591670.0   
8         AB           48          Prairies    6.6   661848.0   642317.0   
9         BC           59  British Columbia    9.5   944735.0   925186.0   
10        YT           60       Territories    4.8   482443.0   474391.0   
11        NT           61       Territories   13.5  1346106.0  1183085.0   
12        NU           62       Territories   21.0  2093190.0  1936113.0   
13       NaN            1            Canada    NaN        NaN        NaN   
14       NaN            4            Canada    NaN        NaN        NaN   
15       NaN            6            Canada    NaN        NaN        NaN   

    Freshwater_skm  
0          31340.0  
1              NaN  
2           1946.0  
3           1458.0  
4         176928.0  
5         158654.0  
6          94241.0  
7          59366.0  
8          19531.0  
9          19549.0  
10          8052.0  
11        163021.0  
12        157077.0  
13             NaN  
14             NaN  
15             NaN

In [418]:
region_combined_regiongroup = region_combined.groupby("GeographicRegion").sum()
region_combined_regiongroup.head(len(region_combined_regiongroup))

Name_EN  \
GeographicRegion                                                      
Atlantic          Newfoundland and LabradorPrince Edward IslandN...   
British Columbia                                   British Columbia   
Canada                                  AtlanticPrairiesTerritories   
Ontario                                                     Ontario   
Prairies                                ManitobaSaskatchewanAlberta   
Quebec                                                       Quebec   
Territories                       YukonNorthwest TerritoriesNunavut   

                                                            Name_FR  \
GeographicRegion                                                      
Atlantic          Terre-Neuve-et-LabradorÎle-du-Prince-ÉdouardNo...   
British Columbia                               Colombie-Britannique   
Canada                                                            0   
Ontario                                                     Ontario   
Prairies                                ManitobaSaskatchewanAlberta   
Quebec                                                       Québec   
Territories                   YukonTerritoires du Nord-OuestNunavut   

                           Abbrev_EN                   Abbrev_FR AlphaCode  \
GeographicRegion                                                             
Atlantic          N.L.P.E.I.N.S.N.B.  T.-N.-L.Î.-P.-É.N.-É.N.-B.  NLPENSNB   
British Columbia                B.C.                       C.-B.        BC   
Canada                             0                           0         0   
Ontario                         Ont.                        Ont.        ON   
Prairies              Man.Sask.Alta.               Man.Sask.Alb.    MBSKAB   
Quebec                          Que.                          Qc        QC   
Territories           Y.T.N.W.T.Nvt.                 YnT.N.-O.Nt    YTNTNU   

                  NumericCode  Area%   Area_skm   Land_skm  Freshwater_skm  
GeographicRegion                                                            
Atlantic                   46    5.5   539064.0   504320.0         34744.0  
British Columbia           59    9.5   944735.0   925186.0         19549.0  
Canada                     11    0.0        0.0        0.0             0.0  
Ontario                    35   10.8  1076395.0   917741.0        158654.0  
Prairies                  141   19.6  1960681.0  1787543.0        173138.0  
Quebec                     24   15.4  1542056.0  1365128.0        176928.0  
Territories               183   39.3  3921739.0  3593589.0        328150.0

In [419]:
region_combined_regiongroup = region_combined_regiongroup.drop(columns=['Name_EN', 'Name_FR', 'Abbrev_EN', 'Abbrev_FR', 'AlphaCode', 'NumericCode'])
region_combined_regiongroup.head()

Area%   Area_skm   Land_skm  Freshwater_skm
GeographicRegion                                             
Atlantic            5.5   539064.0   504320.0         34744.0
British Columbia    9.5   944735.0   925186.0         19549.0
Canada              0.0        0.0        0.0             0.0
Ontario            10.8  1076395.0   917741.0        158654.0
Prairies           19.6  1960681.0  1787543.0        173138.0

In [420]:
region_combined_regiongroup_i = region_combined_regiongroup.reset_index()
region_combined_regiongroup_i.head()

GeographicRegion  Area%   Area_skm   Land_skm  Freshwater_skm
0          Atlantic    5.5   539064.0   504320.0         34744.0
1  British Columbia    9.5   944735.0   925186.0         19549.0
2            Canada    0.0        0.0        0.0             0.0
3           Ontario   10.8  1076395.0   917741.0        158654.0
4          Prairies   19.6  1960681.0  1787543.0        173138.0

In [421]:
region_combined_regiongroup_i = region_combined_regiongroup_i.rename(columns={"GeographicRegion": 'Name_EN'})

region_combined[['Area%', 'Area_skm', 'Land_skm', 'Freshwater_skm']] = [
    region_combined_regiongroup_i[region_combined_regiongroup_i['Name_EN'] == name][['Area%', 'Area_skm', 'Land_skm', 'Freshwater_skm']].iloc[0]
    if name in region_combined_regiongroup_i['Name_EN'].values
    else region_combined[region_combined['Name_EN'] == name][['Area%', 'Area_skm', 'Land_skm', 'Freshwater_skm']].iloc[0]
    for name in region_combined['Name_EN']    
]

region_combined.head(20)

Name_EN                    Name_FR Abbrev_EN Abbrev_FR  \
0   Newfoundland and Labrador    Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1        Prince Edward Island      Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                 Nova Scotia            Nouvelle-Écosse      N.S.     N.-É.   
3               New Brunswick          Nouveau-Brunswick      N.B.     N.-B.   
4                      Quebec                     Québec      Que.        Qc   
5                     Ontario                    Ontario      Ont.      Ont.   
6                    Manitoba                   Manitoba      Man.      Man.   
7                Saskatchewan               Saskatchewan     Sask.     Sask.   
8                     Alberta                    Alberta     Alta.      Alb.   
9            British Columbia       Colombie-Britannique      B.C.     C.-B.   
10                      Yukon                      Yukon      Y.T.        Yn   
11      Northwest Territories  Territoires du Nord-Ouest    N.W.T.   T.N.-O.   
12                    Nunavut                    Nunavut      Nvt.        Nt   
13                   Atlantic                        NaN       NaN       NaN   
14                   Prairies                        NaN       NaN       NaN   
15                Territories                        NaN       NaN       NaN   

   AlphaCode  NumericCode  GeographicRegion  Area%   Area_skm   Land_skm  \
0         NL           10          Atlantic    4.1   405212.0   373872.0   
1         PE           11          Atlantic    0.1     5660.0     5660.0   
2         NS           12          Atlantic    0.6    55284.0    53338.0   
3         NB           13          Atlantic    0.7    72908.0    71450.0   
4         QC           24            Quebec   15.4  1542056.0  1365128.0   
5         ON           35           Ontario   10.8  1076395.0   917741.0   
6         MB           46          Prairies    6.5   647797.0   553556.0   
7         SK           47          Prairies    6.5   651036.0   591670.0   
8         AB           48          Prairies    6.6   661848.0   642317.0   
9         BC           59  British Columbia    9.5   944735.0   925186.0   
10        YT           60       Territories    4.8   482443.0   474391.0   
11        NT           61       Territories   13.5  1346106.0  1183085.0   
12        NU           62       Territories   21.0  2093190.0  1936113.0   
13       NaN            1            Canada    5.5   539064.0   504320.0   
14       NaN            4            Canada   19.6  1960681.0  1787543.0   
15       NaN            6            Canada   39.3  3921739.0  3593589.0   

    Freshwater_skm  
0          31340.0  
1              NaN  
2           1946.0  
3           1458.0  
4         176928.0  
5         158654.0  
6          94241.0  
7          59366.0  
8          19531.0  
9          19549.0  
10          8052.0  
11        163021.0  
12        157077.0  
13         34744.0  
14        173138.0  
15        328150.0

In [5]:
region_pop = pd.read_csv("./datasets/raw/region_data/region_population.csv")
region_pop.head(14)

Geography     Q1 2000     Q2 2000     Q3 2000     Q4 2000  \
0                      Canada  30,525,872  30,594,030  30,685,730  30,783,969   
1   Newfoundland and Labrador     531,774     529,574     527,966     526,732   
2        Prince Edward Island     136,442     136,289     136,470     136,400   
3                 Nova Scotia     934,594     934,291     933,821     934,459   
4               New Brunswick     750,786     750,543     750,517     750,252   
5                      Quebec   7,340,269   7,347,179   7,356,951   7,368,772   
6                     Ontario  11,576,994  11,621,255  11,683,290  11,748,348   
7                    Manitoba   1,144,428   1,145,873   1,147,313   1,148,183   
8                Saskatchewan   1,011,141   1,009,177   1,007,565   1,006,027   
9                     Alberta   2,974,517   2,988,465   3,004,198   3,016,947   
10           British Columbia   4,026,664   4,033,319   4,039,230   4,049,297   
11                      Yukon      30,495      30,382      30,431      30,294   
12      Northwest Territories      40,623      40,455      40,480      40,575   
13                    Nunavut      27,145      27,228      27,498      27,683   

       Q1 2001     Q2 2001     Q3 2001     Q4 2001     Q1 2002  ...  \
0   30,824,441  30,910,996  31,020,855  31,128,873  31,168,896  ...   
1      525,299     523,235     522,043     521,583     521,414  ...   
2      136,377     136,499     136,667     136,876     136,858  ...   
3      933,463     932,909     932,483     933,318     933,757  ...   
4      749,715     749,789     749,823     749,298     748,639  ...   
5    7,373,978   7,383,844   7,396,014   7,411,084   7,417,456  ...   
6   11,771,945  11,827,345  11,897,473  11,961,546  11,979,906  ...   
7    1,148,456   1,149,684   1,151,451   1,152,144   1,152,828  ...   
8    1,003,469   1,001,643   1,000,307     999,241     998,742  ...   
9    3,027,941   3,041,238   3,058,554   3,078,756   3,092,506  ...   
10   4,055,229   4,066,132   4,076,896   4,085,595   4,087,124  ...   
11      30,147      30,124      30,160      30,079      30,156  ...   
12      40,624      40,625      40,850      41,175      41,176  ...   
13      27,798      27,929      28,134      28,178      28,334  ...   

       Q3 2021     Q4 2021     Q1 2022     Q2 2022     Q3 2022     Q4 2022  \
0   38,239,864  38,451,454  38,567,576  38,683,567  38,939,056  39,276,140   
1      527,056     528,335     528,977     529,249     531,583     533,444   
2      162,133     163,775     164,195     165,524     167,188     169,146   
3      999,908   1,006,805   1,010,460   1,014,827   1,025,445   1,035,666   
4      790,802     795,755     798,656     801,778     809,568     817,766   
5    8,572,020   8,603,553   8,613,999   8,627,524   8,672,185   8,730,868   
6   14,842,488  14,942,272  14,999,441  15,046,211  15,145,006  15,283,961   
7    1,391,924   1,396,460   1,400,663   1,405,197   1,413,409   1,423,596   
8    1,167,711   1,168,901   1,171,031   1,173,366   1,178,422   1,187,607   
9    4,431,531   4,450,442   4,465,537   4,480,956   4,510,891   4,561,350   
10   5,226,665   5,267,184   5,286,478   5,310,164   5,356,284   5,403,528   
11      42,961      43,119      43,063      43,454      43,905      44,113   
12      44,579      44,621      44,791      44,828      44,685      44,664   
13      40,086      40,232      40,285      40,489      40,485      40,431   

       Q1 2023     Q2 2023     Q3 2023     Q4 2023  
0   39,498,018  39,739,633  40,097,761  40,528,396  
1      534,964     536,291     538,605     540,418  
2      169,935     171,790     173,787     175,853  
3    1,040,195   1,047,803   1,058,694   1,066,416  
4      820,943     826,622     834,691     842,725  
5    8,767,038   8,814,007   8,874,683   8,948,540  
6   15,378,179  15,457,075  15,608,369  15,801,768  
7    1,432,674   1,443,875   1,454,902   1,465,440  
8    1,194,869   1,200,540   1,209,107   1,218,976  
9    4,598,444   4,645,229   4,695,29

In [423]:
region_pop.dtypes

Geography    object
Q1 2000      object
Q2 2000      object
Q3 2000      object
Q4 2000      object
              ...  
Q4 2022      object
Q1 2023      object
Q2 2023      object
Q3 2023      object
Q4 2023      object
Length: 97, dtype: object

In [424]:
# changing values to ints

for col in region_pop.columns:
    if not(col == 'Geography'):
        region_pop[col] = [
            record.replace(',','')
            for record in region_pop[col]
            ]
        region_pop[col] = pd.to_numeric(region_pop[col])

region_pop.dtypes

Geography    object
Q1 2000       int64
Q2 2000       int64
Q3 2000       int64
Q4 2000       int64
              ...  
Q4 2022       int64
Q1 2023       int64
Q2 2023       int64
Q3 2023       int64
Q4 2023       int64
Length: 97, dtype: object

In [425]:
region_pop.head()

Geography   Q1 2000   Q2 2000   Q3 2000   Q4 2000  \
0                     Canada  30525872  30594030  30685730  30783969   
1  Newfoundland and Labrador    531774    529574    527966    526732   
2       Prince Edward Island    136442    136289    136470    136400   
3                Nova Scotia    934594    934291    933821    934459   
4              New Brunswick    750786    750543    750517    750252   

    Q1 2001   Q2 2001   Q3 2001   Q4 2001   Q1 2002  ...   Q3 2021   Q4 2021  \
0  30824441  30910996  31020855  31128873  31168896  ...  38239864  38451454   
1    525299    523235    522043    521583    521414  ...    527056    528335   
2    136377    136499    136667    136876    136858  ...    162133    163775   
3    933463    932909    932483    933318    933757  ...    999908   1006805   
4    749715    749789    749823    749298    748639  ...    790802    795755   

    Q1 2022   Q2 2022   Q3 2022   Q4 2022   Q1 2023   Q2 2023   Q3 2023  \
0  38567576  38683567  38939056  39276140  39498018  39739633  40097761   
1    528977    529249    531583    533444    534964    536291    538605   
2    164195    165524    167188    169146    169935    171790    173787   
3   1010460   1014827   1025445   1035666   1040195   1047803   1058694   
4    798656    801778    809568    817766    820943    826622    834691   

    Q4 2023  
0  40528396  
1    540418  
2    175853  
3   1066416  
4    842725  

[5 rows x 97 columns]

In [426]:
# adding aggregates for "Atlantic", "Prairies", "Territories"

region_pop_atlantic = region_pop.query(
    'Geography == "New Brunswick" | Geography == "Nova Scotia" | Geography == "Prince Edward Island" | Geography == "Newfoundland and Labrador"')
region_pop.loc[len(region_pop.index)] = ["Atlantic"] + list(region_pop_atlantic.sum(numeric_only=True))

region_pop_prairies = region_pop.query(
    'Geography == "Manitoba" | Geography == "Alberta" | Geography == "Saskatchewan"')
region_pop.loc[len(region_pop.index)] = ["Prairies"] + list(region_pop_prairies.sum(numeric_only=True))

region_pop_territories = region_pop.query(
    'Geography == "Yukon" | Geography == "Northwest Territories" | Geography == "Nunavut"')
region_pop.loc[len(region_pop.index)] = ["Territories"] + list(region_pop_territories.sum(numeric_only=True))

region_pop.head(20)


Geography   Q1 2000   Q2 2000   Q3 2000   Q4 2000  \
0                      Canada  30525872  30594030  30685730  30783969   
1   Newfoundland and Labrador    531774    529574    527966    526732   
2        Prince Edward Island    136442    136289    136470    136400   
3                 Nova Scotia    934594    934291    933821    934459   
4               New Brunswick    750786    750543    750517    750252   
5                      Quebec   7340269   7347179   7356951   7368772   
6                     Ontario  11576994  11621255  11683290  11748348   
7                    Manitoba   1144428   1145873   1147313   1148183   
8                Saskatchewan   1011141   1009177   1007565   1006027   
9                     Alberta   2974517   2988465   3004198   3016947   
10           British Columbia   4026664   4033319   4039230   4049297   
11                      Yukon     30495     30382     30431     30294   
12      Northwest Territories     40623     40455     40480     40575   
13                    Nunavut     27145     27228     27498     27683   
14                   Atlantic   2353596   2350697   2348774   2347843   
15                   Prairies   5130086   5143515   5159076   5171157   
16                Territories     98263     98065     98409     98552   

     Q1 2001   Q2 2001   Q3 2001   Q4 2001   Q1 2002  ...   Q3 2021   Q4 2021  \
0   30824441  30910996  31020855  31128873  31168896  ...  38239864  38451454   
1     525299    523235    522043    521583    521414  ...    527056    528335   
2     136377    136499    136667    136876    136858  ...    162133    163775   
3     933463    932909    932483    933318    933757  ...    999908   1006805   
4     749715    749789    749823    749298    748639  ...    790802    795755   
5    7373978   7383844   7396014   7411084   7417456  ...   8572020   8603553   
6   11771945  11827345  11897473  11961546  11979906  ...  14842488  14942272   
7    1148456   1149684   1151451   1152144   1152828  ...   1391924   1396460   
8    1003469   1001643   1000307    999241    998742  ...   1167711   1168901   
9    3027941   3041238   3058554   3078756   3092506  ...   4431531   4450442   
10   4055229   4066132   4076896   4085595   4087124  ...   5226665   5267184   
11     30147     30124     30160     30079     30156  ...     42961     43119   
12     40624     40625     40850     41175     41176  ...     44579     44621   
13     27798     27929     28134     28178     28334  ...     40086     40232   
14   2344854   2342432   2341016   2341075   2340668  ...   2479899   2494670   
15   5179866   5192565   5210312   5230141   5244076  ...   6991166   7015803   
16     98569     98678     99144     99432     99666  ...    127626    127972   

     Q1 2022   Q2 2022   Q3 2022   Q4 2022   Q1 2023   Q2 2023   Q3 2023  \
0   38567576  38683567  38939056  39276140  39498018  39739633  40097761   
1     528977    529249    531583    533444    534964    536291    538605   
2     164195    165524    167188    169146    169935    171790    173787   
3    1010460   1014827   1025445   1035666   1040195   1047803   1058694   
4     798656    801778    809568    817766    820943    826622    834691   
5    8613999   8627524   8672185   8730868   8767038   8814007   8874683   
6   14999441  15046211  15145006  15283961  15378179  15457075  15608369   
7    1400663   1405197   1413409   1423596   1432674   1443875   1454902   
8    1171031   1173366   1178422   1187607   1194869   1200540   1209107   
9    4465537   4480956   4510891   4561350   4598444   4645229   4695290   
10   5286478   5310164   5356284   5403528   5431355   5466646   5519013   
11     43063     43454     43905     44113     44386     44596     44975   
12     44791     44828     44685     44664     44515     44678     44972   
13     40285     40489     40485     40431     40521     40481     40673   
14   2502288   2511378   2533784   2556022   2566037   2582506   2605777   
15   7037231   7059519   71027

In [427]:
# {0: “less than 1 million people”,  
# 1: “between 1 and 5 million people”, 
# 2: “between 5 to 10 million people”, 
# 3: “between 10 to 15 million people”, 
# 4: “over 15 million people”}

bins = [0, 1000000, 5000000, 10000000, 15000000, 50000000]

In [428]:
for col in region_pop.columns:
    if not(col == 'Geography'):
        region_pop[col] = pd.cut(region_pop[col], bins=bins)

region_pop.head(20)

Geography               Q1 2000               Q2 2000  \
0                      Canada  (15000000, 50000000]  (15000000, 50000000]   
1   Newfoundland and Labrador          (0, 1000000]          (0, 1000000]   
2        Prince Edward Island          (0, 1000000]          (0, 1000000]   
3                 Nova Scotia          (0, 1000000]          (0, 1000000]   
4               New Brunswick          (0, 1000000]          (0, 1000000]   
5                      Quebec   (5000000, 10000000]   (5000000, 10000000]   
6                     Ontario  (10000000, 15000000]  (10000000, 15000000]   
7                    Manitoba    (1000000, 5000000]    (1000000, 5000000]   
8                Saskatchewan    (1000000, 5000000]    (1000000, 5000000]   
9                     Alberta    (1000000, 5000000]    (1000000, 5000000]   
10           British Columbia    (1000000, 5000000]    (1000000, 5000000]   
11                      Yukon          (0, 1000000]          (0, 1000000]   
12      Northwest Territories          (0, 1000000]          (0, 1000000]   
13                    Nunavut          (0, 1000000]          (0, 1000000]   
14                   Atlantic    (1000000, 5000000]    (1000000, 5000000]   
15                   Prairies   (5000000, 10000000]   (5000000, 10000000]   
16                Territories          (0, 1000000]          (0, 1000000]   

                 Q3 2000               Q4 2000               Q1 2001  \
0   (15000000, 50000000]  (15000000, 50000000]  (15000000, 50000000]   
1           (0, 1000000]          (0, 1000000]          (0, 1000000]   
2           (0, 1000000]          (0, 1000000]          (0, 1000000]   
3           (0, 1000000]          (0, 1000000]          (0, 1000000]   
4           (0, 1000000]          (0, 1000000]          (0, 1000000]   
5    (5000000, 10000000]   (5000000, 10000000]   (5000000, 10000000]   
6   (10000000, 15000000]  (10000000, 15000000]  (10000000, 15000000]   
7     (1000000, 5000000]    (1000000, 5000000]    (1000000, 5000000]   
8     (1000000, 5000000]    (1000000, 5000000]    (1000000, 5000000]   
9     (1000000, 5000000]    (1000000, 5000000]    (1000000, 5000000]   
10    (1000000, 5000000]    (1000000, 5000000]    (1000000, 5000000]   
11          (0, 1000000]          (0, 1000000]          (0, 1000000]   
12          (0, 1000000]          (0, 1000000]          (0, 1000000]   
13          (0, 1000000]          (0, 1000000]          (0, 1000000]   
14    (1000000, 5000000]    (1000000, 5000000]    (1000000, 5000000]   
15   (5000000, 10000000]   (5000000, 10000000]   (5000000, 10000000]   
16          (0, 1000000]          (0, 1000000]          (0, 1000000]   

                 Q2 2001               Q3 2001               Q4 2001  \
0   (15000000, 50000000]  (15000000, 50000000]  (15000000, 50000000]   
1           (0, 1000000]          (0, 1000000]          (0, 1000000]   
2           (0, 1000000]          (0, 1000000]          (0, 1000000]   
3           (0, 1000000]          (0, 1000000]          (0, 1000000]   
4           (0, 1000000]          (0, 1000000]          (0, 1000000]   
5    (5000000, 10000000]   (5000000, 10000000]   (5000000, 10000000]   
6   (10000000, 15000000]  (10000000, 15000000]  (10000000, 15000000]   
7     (1000000, 5000000]    (1000000, 5000000]    (1000000, 5000000]   
8     (1000000, 5000000]    (1000000, 5000000]          (0, 1000000]   
9     (1000000, 5000000]    (1000000, 5000000]    (1000000, 5000000]   
10    (1000000, 5000000]    (1000000, 5000000]    (1000000, 5000000]   
11          (0, 1000000]          (0, 1000000]          (0, 1000000]   
12          (0, 1000000]          (0, 1000000]          (0, 1000000]   
13          (0, 1000000]          (0, 1000000]          (0, 1000000]   
14    (1000000, 5000000]    (1000000, 5000000]    (1000000, 5000000]   
15   (5000000, 10000000]   (5000000, 10000000]   (5000000, 10000000]   
16          (0, 1000000]          (0, 1000000]          (0, 1000000]   

                 Q1 2002  ...          

In [429]:
# region_pop_change = pd.DataFrame()
region_pop_on = region_pop[region_pop['Geography'] == 'Ontario']
region_pop_on.head()


Geography               Q1 2000               Q2 2000               Q3 2000  \
6   Ontario  (10000000, 15000000]  (10000000, 15000000]  (10000000, 15000000]   

                Q4 2000               Q1 2001               Q2 2001  \
6  (10000000, 15000000]  (10000000, 15000000]  (10000000, 15000000]   

                Q3 2001               Q4 2001               Q1 2002  ...  \
6  (10000000, 15000000]  (10000000, 15000000]  (10000000, 15000000]  ...   

                Q3 2021               Q4 2021               Q1 2022  \
6  (10000000, 15000000]  (10000000, 15000000]  (10000000, 15000000]   

                Q2 2022               Q3 2022               Q4 2022  \
6  (15000000, 50000000]  (15000000, 50000000]  (15000000, 50000000]   

                Q1 2023               Q2 2023               Q3 2023  \
6  (15000000, 50000000]  (15000000, 50000000]  (15000000, 50000000]   

                Q4 2023  
6  (15000000, 50000000]  

[1 rows x 97 columns]

In [431]:
region_pop[region_pop['Geography'] == 'Ontario'].dtypes

Geography      object
Q1 2000      category
Q2 2000      category
Q3 2000      category
Q4 2000      category
               ...   
Q4 2022      category
Q1 2023      category
Q2 2023      category
Q3 2023      category
Q4 2023      category
Length: 97, dtype: object

In [432]:
region_pop_changes = {}

for region in region_pop['Geography'].values:
    region_pop_only = region_pop[region_pop['Geography'] == region]
    region_pop_specific = region_pop_only.set_index('Geography').T.drop_duplicates(subset=[region]).T
    region_pop_specific = region_pop_specific.reset_index()
    region_pop_changes[region] = region_pop_specific

display(region_pop_changes['Ontario'])

Geography               Q1 2000               Q2 2022
0   Ontario  (10000000, 15000000]  (15000000, 50000000]

In [433]:
display(region_pop_changes['British Columbia'])

Geography             Q1 2000              Q3 2018
0  British Columbia  (1000000, 5000000]  (5000000, 10000000]

Attempting to get a table in the following format:
    Geography           |   EffectiveDate   |   Population
    British Columbia    |   2000-01-01      |       1
    British Columbia    |   2018-09-01      |       2

In [434]:
print(list(region_pop_changes.keys()))
# ['Canada', 'Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia', 
# 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta', 
# 'British Columbia', 'Yukon', 'Northwest Territories', 'Nunavut',
# , 'Atlantic', 'Prairies', 'Territories']

['Canada', 'Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories', 'Nunavut', 'Atlantic', 'Prairies', 'Territories']


In [435]:
region_pop_changes['British Columbia']['Q1 2000'][0]

Interval(1000000, 5000000, closed='right')

In [436]:
regions_pop_list = list(region_pop_changes.keys())
region_pop_changes_formatted = []

for key in regions_pop_list:
    if not(key == 'Canada'):
        for col in region_pop_changes[key]:
            if not(col == 'Geography'):
                region_pop_changes_formatted.append({
                    'Geography': key,
                    'EffectiveDate': col,
                    'Population': region_pop_changes[key][col][0]
                })

region_pop_changes_formatted = pd.DataFrame(region_pop_changes_formatted)
display(region_pop_changes_formatted.head())

Geography EffectiveDate          Population
0  Newfoundland and Labrador       Q1 2000        (0, 1000000]
1       Prince Edward Island       Q1 2000        (0, 1000000]
2                Nova Scotia       Q1 2000        (0, 1000000]
3                Nova Scotia       Q4 2021  (1000000, 5000000]
4              New Brunswick       Q1 2000        (0, 1000000]

In [437]:
display(region_pop_changes_formatted[region_pop_changes_formatted['Geography'] == 'British Columbia'])

Geography EffectiveDate           Population
12  British Columbia       Q1 2000   (1000000, 5000000]
13  British Columbia       Q3 2018  (5000000, 10000000]

In [440]:
region_pop_changes_formatted[['Population_Lower', 'Population_Upper']] = [[x.left, x.right] for x in region_pop_changes_formatted['Population']]
region_pop_changes_formatted = region_pop_changes_formatted.drop(columns=['Population'])
display(region_pop_changes_formatted[region_pop_changes_formatted['Geography'] == 'British Columbia'])

Geography EffectiveDate  Population_Lower  Population_Upper
12  British Columbia       Q1 2000           1000000           5000000
13  British Columbia       Q3 2018           5000000          10000000

Need to convert EffectiveDate to Datetime

In [7]:
date_lookup = pd.read_csv("./datasets/dates_dim.csv")
date_lookup.head()

Date_ID        Date Month_LongName Month_ShortName  Month  \
0        0  2001-01-01        January             Jan      1   
1        1  2001-02-01       February             Feb      2   
2        2  2001-03-01          March             Mar      3   
3        3  2001-04-01          April             Apr      4   
4        4  2001-05-01            May             May      5   

   NumberOfDaysInMonth  Quarter  NumberOfDaysInQuarter  Year  \
0                   31        1                     90  2001   
1                   28        1                     90  2001   
2                   31        1                     90  2001   
3                   30        2                     91  2001   
4                   31        2                     91  2001   

   NumberOfDaysInYear  LeapYear  
0                 365         0  
1                 365         0  
2                 365         0  
3                 365         0  
4                 365         0

In [442]:
region_pop_changes_formatted['EffectiveDate'] = [
    date_lookup[date_lookup['CalendarQuarter'] == int(qd[1])][date_lookup['CalendarYear'] == int(qd[3:])]['Date'].iloc[0]
    if int(qd[3:]) > 2001
    else '2001-01-01'
    for qd in region_pop_changes_formatted['EffectiveDate']
]

display(region_pop_changes_formatted.head())

C:\Users\svetl\AppData\Local\Temp\ipykernel_4620\135889141.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  date_lookup[date_lookup['CalendarQuarter'] == int(qd[1])][date_lookup['CalendarYear'] == int(qd[3:])]['Date'].iloc[0]
C:\Users\svetl\AppData\Local\Temp\ipykernel_4620\135889141.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  date_lookup[date_lookup['CalendarQuarter'] == int(qd[1])][date_lookup['CalendarYear'] == int(qd[3:])]['Date'].iloc[0]
C:\Users\svetl\AppData\Local\Temp\ipykernel_4620\135889141.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  date_lookup[date_lookup['CalendarQuarter'] == int(qd[1])][date_lookup['CalendarYear'] == int(qd[3:])]['Date'].iloc[0]


Geography EffectiveDate  Population_Lower  Population_Upper
0  Newfoundland and Labrador    2001-01-01                 0           1000000
1       Prince Edward Island    2001-01-01                 0           1000000
2                Nova Scotia    2001-01-01                 0           1000000
3                Nova Scotia    2021-10-01           1000000           5000000
4              New Brunswick    2001-01-01                 0           1000000

In [443]:
display(region_pop_changes_formatted[region_pop_changes_formatted['Geography'] == 'British Columbia'])

Geography EffectiveDate  Population_Lower  Population_Upper
12  British Columbia    2001-01-01           1000000           5000000
13  British Columbia    2018-07-01           5000000          10000000

In [444]:
region_pop_changes_formatted.dtypes

Geography           object
EffectiveDate       object
Population_Lower     int64
Population_Upper     int64
dtype: object

In [445]:
region_pop_changes_formatted['EffectiveDate'] = pd.to_datetime(region_pop_changes_formatted['EffectiveDate'])
region_pop_changes_formatted.dtypes

Geography                   object
EffectiveDate       datetime64[ns]
Population_Lower             int64
Population_Upper             int64
dtype: object

In [446]:
region_pop_changes_formatted.isna().sum()

Geography           0
EffectiveDate       0
Population_Lower    0
Population_Upper    0
dtype: int64

combining with the rest of the region dimension

In [447]:
region_combined.head()

Name_EN                  Name_FR Abbrev_EN Abbrev_FR  \
0  Newfoundland and Labrador  Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1       Prince Edward Island    Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                Nova Scotia          Nouvelle-Écosse      N.S.     N.-É.   
3              New Brunswick        Nouveau-Brunswick      N.B.     N.-B.   
4                     Quebec                   Québec      Que.        Qc   

  AlphaCode  NumericCode GeographicRegion  Area%   Area_skm   Land_skm  \
0        NL           10         Atlantic    4.1   405212.0   373872.0   
1        PE           11         Atlantic    0.1     5660.0     5660.0   
2        NS           12         Atlantic    0.6    55284.0    53338.0   
3        NB           13         Atlantic    0.7    72908.0    71450.0   
4        QC           24           Quebec   15.4  1542056.0  1365128.0   

   Freshwater_skm  
0         31340.0  
1             NaN  
2          1946.0  
3          1458.0  
4        176928.0

In [448]:
region_combined_with_pop = region_combined.merge(region_pop_changes_formatted, how="left", left_on='Name_EN', right_on='Geography')
region_combined_with_pop.head()

Name_EN                  Name_FR Abbrev_EN Abbrev_FR  \
0  Newfoundland and Labrador  Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1       Prince Edward Island    Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                Nova Scotia          Nouvelle-Écosse      N.S.     N.-É.   
3                Nova Scotia          Nouvelle-Écosse      N.S.     N.-É.   
4              New Brunswick        Nouveau-Brunswick      N.B.     N.-B.   

  AlphaCode  NumericCode GeographicRegion  Area%  Area_skm  Land_skm  \
0        NL           10         Atlantic    4.1  405212.0  373872.0   
1        PE           11         Atlantic    0.1    5660.0    5660.0   
2        NS           12         Atlantic    0.6   55284.0   53338.0   
3        NS           12         Atlantic    0.6   55284.0   53338.0   
4        NB           13         Atlantic    0.7   72908.0   71450.0   

   Freshwater_skm                  Geography EffectiveDate  Population_Lower  \
0         31340.0  Newfoundland and Labrador    2001-01-01                 0   
1             NaN       Prince Edward Island    2001-01-01                 0   
2          1946.0                Nova Scotia    2001-01-01                 0   
3          1946.0                Nova Scotia    2021-10-01           1000000   
4          1458.0              New Brunswick    2001-01-01                 0   

   Population_Upper  
0           1000000  
1           1000000  
2           1000000  
3           5000000  
4           1000000

In [449]:
region_combined_with_pop[region_combined_with_pop['Name_EN'] == 'British Columbia']

Name_EN               Name_FR Abbrev_EN Abbrev_FR AlphaCode  \
12  British Columbia  Colombie-Britannique      B.C.     C.-B.        BC   
13  British Columbia  Colombie-Britannique      B.C.     C.-B.        BC   

    NumericCode  GeographicRegion  Area%  Area_skm  Land_skm  Freshwater_skm  \
12           59  British Columbia    9.5  944735.0  925186.0         19549.0   
13           59  British Columbia    9.5  944735.0  925186.0         19549.0   

           Geography EffectiveDate  Population_Lower  Population_Upper  
12  British Columbia    2001-01-01           1000000           5000000  
13  British Columbia    2018-07-01           5000000          10000000

In [450]:
region_combined_with_pop = region_combined_with_pop.drop(columns=['Geography'])
display(region_combined_with_pop.head())
display(region_combined_with_pop.dtypes)

Name_EN                  Name_FR Abbrev_EN Abbrev_FR  \
0  Newfoundland and Labrador  Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1       Prince Edward Island    Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                Nova Scotia          Nouvelle-Écosse      N.S.     N.-É.   
3                Nova Scotia          Nouvelle-Écosse      N.S.     N.-É.   
4              New Brunswick        Nouveau-Brunswick      N.B.     N.-B.   

  AlphaCode  NumericCode GeographicRegion  Area%  Area_skm  Land_skm  \
0        NL           10         Atlantic    4.1  405212.0  373872.0   
1        PE           11         Atlantic    0.1    5660.0    5660.0   
2        NS           12         Atlantic    0.6   55284.0   53338.0   
3        NS           12         Atlantic    0.6   55284.0   53338.0   
4        NB           13         Atlantic    0.7   72908.0   71450.0   

   Freshwater_skm EffectiveDate  Population_Lower  Population_Upper  
0         31340.0    2001-01-01                 0           1000000  
1             NaN    2001-01-01                 0           1000000  
2          1946.0    2001-01-01                 0           1000000  
3          1946.0    2021-10-01           1000000           5000000  
4          1458.0    2001-01-01                 0           1000000

Name_EN                     object
Name_FR                     object
Abbrev_EN                   object
Abbrev_FR                   object
AlphaCode                   object
NumericCode                  int64
GeographicRegion            object
Area%                      float64
Area_skm                   float64
Land_skm                   float64
Freshwater_skm             float64
EffectiveDate       datetime64[ns]
Population_Lower             int64
Population_Upper             int64
dtype: object

In [452]:
# region_combined_with_pop['Population'] = region_combined_with_pop['Population'].fillna(-1)
# region_combined_with_pop = region_combined_with_pop.astype({'Population':'int'})
region_combined_with_pop.dtypes

Name_EN                     object
Name_FR                     object
Abbrev_EN                   object
Abbrev_FR                   object
AlphaCode                   object
NumericCode                  int64
GeographicRegion            object
Area%                      float64
Area_skm                   float64
Land_skm                   float64
Freshwater_skm             float64
EffectiveDate       datetime64[ns]
Population_Lower             int64
Population_Upper             int64
dtype: object

In [453]:
region_combined_with_pop.head(30)

Name_EN                    Name_FR Abbrev_EN Abbrev_FR  \
0   Newfoundland and Labrador    Terre-Neuve-et-Labrador      N.L.  T.-N.-L.   
1        Prince Edward Island      Île-du-Prince-Édouard    P.E.I.  Î.-P.-É.   
2                 Nova Scotia            Nouvelle-Écosse      N.S.     N.-É.   
3                 Nova Scotia            Nouvelle-Écosse      N.S.     N.-É.   
4               New Brunswick          Nouveau-Brunswick      N.B.     N.-B.   
5                      Quebec                     Québec      Que.        Qc   
6                     Ontario                    Ontario      Ont.      Ont.   
7                     Ontario                    Ontario      Ont.      Ont.   
8                    Manitoba                   Manitoba      Man.      Man.   
9                Saskatchewan               Saskatchewan     Sask.     Sask.   
10               Saskatchewan               Saskatchewan     Sask.     Sask.   
11                    Alberta                    Alberta     Alta.      Alb.   
12           British Columbia       Colombie-Britannique      B.C.     C.-B.   
13           British Columbia       Colombie-Britannique      B.C.     C.-B.   
14                      Yukon                      Yukon      Y.T.        Yn   
15      Northwest Territories  Territoires du Nord-Ouest    N.W.T.   T.N.-O.   
16                    Nunavut                    Nunavut      Nvt.        Nt   
17                   Atlantic                        NaN       NaN       NaN   
18                   Prairies                        NaN       NaN       NaN   
19                Territories                        NaN       NaN       NaN   

   AlphaCode  NumericCode  GeographicRegion  Area%   Area_skm   Land_skm  \
0         NL           10          Atlantic    4.1   405212.0   373872.0   
1         PE           11          Atlantic    0.1     5660.0     5660.0   
2         NS           12          Atlantic    0.6    55284.0    53338.0   
3         NS           12          Atlantic    0.6    55284.0    53338.0   
4         NB           13          Atlantic    0.7    72908.0    71450.0   
5         QC           24            Quebec   15.4  1542056.0  1365128.0   
6         ON           35           Ontario   10.8  1076395.0   917741.0   
7         ON           35           Ontario   10.8  1076395.0   917741.0   
8         MB           46          Prairies    6.5   647797.0   553556.0   
9         SK           47          Prairies    6.5   651036.0   591670.0   
10        SK           47          Prairies    6.5   651036.0   591670.0   
11        AB           48          Prairies    6.6   661848.0   642317.0   
12        BC           59  British Columbia    9.5   944735.0   925186.0   
13        BC           59  British Columbia    9.5   944735.0   925186.0   
14        YT           60       Territories    4.8   482443.0   474391.0   
15        NT           61       Territories   13.5  1346106.0  1183085.0   
16        NU           62       Territories   21.0  2093190.0  1936113.0   
17       NaN            1            Canada    5.5   539064.0   504320.0   
18       NaN            4            Canada   19.6  1960681.0  1787543.0   
19       NaN            6            Canada   39.3  3921739.0  3593589.0   

    Freshwater_skm EffectiveDate  Population_Lower  Population_Upper  
0          31340.0    2001-01-01                 0           1000000  
1              NaN    2001-01-01                 0           1000000  
2           1946.0    2001-01-01                 0           1000000  
3           1946.0    2021-10-01           1000000           5000000  
4           1458.0    2001-01-01                 0           1000000  
5         176928.0    2001-01-01           5000000          10000000  
6         158654.0    2001-01-01          10000000          15000000  
7         158654.0    2022-04-01          15000000          50000000  
8          94241.0    2001-01-01           1000000           5000000  
9          59366.0    2001-01-01  

In [454]:
region_combined_with_pop["Region_ID"] = range(0, len(region_combined_with_pop))
region_combined_with_pop = region_combined_with_pop[["Region_ID", "Name_EN", "Name_FR", "Abbrev_EN", 
                          "Abbrev_FR", "AlphaCode", "NumericCode", "GeographicRegion",
                          "Area%", "Area_skm", "Land_skm", "Freshwater_skm",
                          "EffectiveDate", "Population_Lower", "Population_Upper"]]

region_combined_with_pop = region_combined_with_pop.rename(columns={
    "Area%": "Area_%",
    "Area_skm": "Area",
    "Land_skm": "Land",
    "Freshwater_skm": "Freshwater"
    })

region_combined_with_pop.head()

Region_ID                    Name_EN                  Name_FR Abbrev_EN  \
0          0  Newfoundland and Labrador  Terre-Neuve-et-Labrador      N.L.   
1          1       Prince Edward Island    Île-du-Prince-Édouard    P.E.I.   
2          2                Nova Scotia          Nouvelle-Écosse      N.S.   
3          3                Nova Scotia          Nouvelle-Écosse      N.S.   
4          4              New Brunswick        Nouveau-Brunswick      N.B.   

  Abbrev_FR AlphaCode  NumericCode GeographicRegion  Area_%      Area  \
0  T.-N.-L.        NL           10         Atlantic     4.1  405212.0   
1  Î.-P.-É.        PE           11         Atlantic     0.1    5660.0   
2     N.-É.        NS           12         Atlantic     0.6   55284.0   
3     N.-É.        NS           12         Atlantic     0.6   55284.0   
4     N.-B.        NB           13         Atlantic     0.7   72908.0   

       Land  Freshwater EffectiveDate  Population_Lower  Population_Upper  
0  373872.0     31340.0    2001-01-01                 0           1000000  
1    5660.0         NaN    2001-01-01                 0           1000000  
2   53338.0      1946.0    2001-01-01                 0           1000000  
3   53338.0      1946.0    2021-10-01           1000000           5000000  
4   71450.0      1458.0    2001-01-01                 0           1000000

In [460]:
region_combined_with_pop = region_combined_with_pop.rename(columns={'Area_%': 'Area_Percentage'})

In [461]:
region_combined_with_pop.to_csv('./datasets/region_dim.csv', index=False)

In [8]:
processed_check = pd.read_csv('./datasets/region_dim.csv')
display(len(processed_check))
display(processed_check.dtypes)
display(processed_check.head())

20

Region_ID             int64
Name_EN              object
Name_FR              object
Abbrev_EN            object
Abbrev_FR            object
AlphaCode            object
NumericCode           int64
GeographicRegion     object
Area_Percentage     float64
Area                float64
Land                float64
Freshwater          float64
EffectiveDate        object
Population_Lower      int64
Population_Upper      int64
dtype: object

Region_ID                    Name_EN                  Name_FR Abbrev_EN  \
0          0  Newfoundland and Labrador  Terre-Neuve-et-Labrador      N.L.   
1          1       Prince Edward Island    Île-du-Prince-Édouard    P.E.I.   
2          2                Nova Scotia          Nouvelle-Écosse      N.S.   
3          3                Nova Scotia          Nouvelle-Écosse      N.S.   
4          4              New Brunswick        Nouveau-Brunswick      N.B.   

  Abbrev_FR AlphaCode  NumericCode GeographicRegion  Area_Percentage  \
0  T.-N.-L.        NL           10         Atlantic              4.1   
1  Î.-P.-É.        PE           11         Atlantic              0.1   
2     N.-É.        NS           12         Atlantic              0.6   
3     N.-É.        NS           12         Atlantic              0.6   
4     N.-B.        NB           13         Atlantic              0.7   

       Area      Land  Freshwater EffectiveDate  Population_Lower  \
0  405212.0  373872.0     31340.0    2001-01-01                 0   
1    5660.0    5660.0         NaN    2001-01-01                 0   
2   55284.0   53338.0      1946.0    2001-01-01                 0   
3   55284.0   53338.0      1946.0    2021-10-01           1000000   
4   72908.0   71450.0      1458.0    2001-01-01                 0   

   Population_Upper  
0           1000000  
1           1000000  
2           1000000  
3           5000000  
4           1000000